## Task 9 - Simulation Environment

### Axis Limits Calculation

The first step in this notebook determines the maximum and minimum values of the robot's working envelope along the X, Y, and Z axes.
The method involves:
- Iterating through each joint of the robot.
- Setting extreme values for the joint positions.
- Recording the pipette's position at each extreme.

The calculated values are then used to define the working envelope.


In [1]:
# Import the required libraries
from sim_class import Simulation
import pybullet as p

# Initialize the simulation with 1 robot
sim = Simulation(num_agents=1, render=False)

# Define a function to test the axis limits of the pipette
def test_axis_limits(sim):
    # Get the ID of the first robot in the simulation
    robot_id = sim.robotIds[0]  
    # Dictionary to store axis limits for X, Y, and Z
    axis_limits = {"x": [], "y": [], "z": []}

    # Test each joint by setting it to extreme values
    for joint_index in range(3): # Loop over 3 joints representing X, Y and Z
        for extreme_value in [-2, 2]:  # Test extreme values
            # Set the joint state to the extreme value
            p.resetJointState(robot_id, joint_index, targetValue=extreme_value)
            # Get the pipette position after applying the joint state
            pipette_position = sim.get_pipette_position(robot_id)
            # Append the pipette's X, Y, Z positions to the respective axis lists
            axis_limits["x"].append(pipette_position[0])
            axis_limits["y"].append(pipette_position[1])
            axis_limits["z"].append(pipette_position[2])

    # Get the min and max values for each axis
    axis_limits = {axis: (min(values), max(values)) for axis, values in axis_limits.items()}
    return axis_limits

# Run the test
limits = test_axis_limits(sim)
print("Axis limits:", limits)

# Close the simulation
sim.close()

Axis limits: {'x': (-1.927, 2.073), 'y': (-1.9105, 2.0895), 'z': (-1.8805, 2.1195)}


### Moving the Pipette to the Corners of the Working Envelope

After determining the working envelope, this section moves the pipette to all 8 corners of the envelope.

Steps:
1. The corners are defined by combining the min and max values of X, Y, and Z.
2. The pipette moves to each corner in sequence, logging its state after reaching each position.
3. The simulation runs for 100 steps at each corner to ensure the movement is completed.

In [3]:
# Initialize the simulation
sim = Simulation(num_agents=1, render=True)  # Render enabled for visualization

# Define the working envelope axis limits we got from the code above
x_limits = [-1.927, 2.073]
y_limits = [-1.9105, 2.0895]
z_limits = [-1.8805, 2.1195]

# Define the 8 corners of the envelope
corners = [
    (x, y, z)
    for x in x_limits
    for y in y_limits
    for z in z_limits
]

# Move to each corner using actions
for i, corner in enumerate(corners):
    x, y, z = corner
    print(f"Moving to corner {i + 1}: X={x}, Y={y}, Z={z}")
    
    # Define actions based on corner coordinates, no drop command
    actions = [[x, y, z, 0]]

    # Run the simulation with the defined action
    sim.run(actions, num_steps=100)
    
    # Log the pipette state
    states = sim.get_states()
    print(f"State at corner {i + 1}: {states}")

# Close the simulation after moving to all corners
sim.close()

Moving to corner 1: X=-1.927, Y=-1.9105, Z=-1.8805
State at corner 1: {'robotId_1': {'joint_states': {'joint_0': {'position': 0.2600024813558601, 'velocity': -0.00021561056528457122, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': 500.00000000000006}, 'joint_1': {'position': 0.2600519610390076, 'velocity': -0.003551240236439791, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': 500.00000000000006}, 'joint_2': {'position': 8.578800652256448e-05, 'velocity': 0.00038479773542643924, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': -800.0}}, 'robot_position': [-0.26000107846934223, -0.2600620780677207, 0.030074771190964446], 'pipette_position': [-0.187, -0.1706, 0.1196]}}
Moving to corner 2: X=-1.927, Y=-1.9105, Z=2.1195
State at corner 2: {'robotId_1': {'joint_states': {'joint_0': {'position': 0.26000036678948396, 'velocity': -3.029514199102178e-05, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': 500.00000000000006},